# Data Cleaning

**Important Note:** The slides in the deck have modified versions of the dataset to illustrate and provide examples for various errors to clean. The real dataset for this project may not have the same errors, so don't expect to find the exact same observations as in the slides.

In [2]:
# First import the libraries you'll need
import pandas as pd
pd.set_option('display.max_columns', 100)

import numpy as np

from matplotlib import pyplot as plt
%matplotlib inline

import seaborn as sns

In [3]:
# import the dataset
df = pd.read_csv('real_estate.csv')
df.tail()

,price,year_sold,property_tax,insurance,beds,baths,sqft,year_built,lot_size,basement,property_type
1877,385000,2002,1250,381,5,6,6381,2004,224334,1,Bunglow
1878,690000,2015,1553,473,5,6,6501,1956,23086,1,Bunglow
1879,600000,1999,942,287,5,6,7064,1995,217800,1,Bunglow
1880,759900,2009,803,245,5,6,7500,2006,8886,1,Bunglow
1881,735000,2015,1459,444,5,6,7515,1958,10497,1,Bunglow


#### Now let's get started with cleaning the data!

In [4]:
df.describe()

,price,year_sold,property_tax,insurance,beds,baths,sqft,year_built,lot_size,basement
count,1882.000000,1882.000000,1882.000000,1882.000000,1882.000000,1882.000000,1882.000000,1882.000000,1882.000000,1882.000000
mean,422908.798618,2007.107864,466.903294,140.486716,3.421360,2.580765,2330.171626,1982.981403,12751.197131,0.880446
std,151473.251553,5.195851,231.653858,72.935379,1.068335,0.945125,1336.926475,20.287099,35304.268897,0.324525
min,200000.000000,1993.000000,88.000000,30.000000,1.000000,1.000000,500.000000,1880.000000,0.000000,0.000000
25%,300000.000000,2004.000000,320.000000,94.000000,3.000000,2.000000,1346.000000,1970.000000,1542.000000,1.000000
50%,392000.000000,2007.000000,426.000000,125.000000,3.000000,3.000000,1907.500000,1986.000000,6074.000000,1.000000
75%,525000.000000,2011.000000,569.000000,169.000000,4.000000,3.000000,3005.000000,2000.000000,11761.000000,1.000000
max,800000.000000,2016.000000,4508.000000,1374.000000,5.000000,6.000000,8450.000000,2015.000000,436471.000000,1.000000


## I. Handle Missing Data

* For example, the `basement` feature? It has values **1** and **NaN**.

In [13]:
# Display unique values of 'basement'
df['basement'].unique()

array([0, 1], dtype=int64)

The **NaN** values are missing values. They actually indicate that the property does not have a `basement`. Therefore, let's replace them with the value **0**.

Using the `.fillna()` function replace **NaN** values with 0. 

In [14]:
# Replace missing basement values to 0
# inplace=True updates the values in the original DataFrame.
#df['basement']= df['basement'].fillna(0)
df['basement'].fillna(0, inplace=True)

In [15]:
# Display unique values of 'basement'
df['basement'].unique()

array([0, 1], dtype=int64)

In [16]:
# change the basement type to integer
df.basement=df.basement.astype(int)

## II. Remove "Guilty" Outliers

Outliers can cause problems with certain types of models. For example, linear regression models are less robust to outliers than decision tree models. If you have a good reason to remove an outlier, it will help your model's performance.

#### A. Plot the violin plots for <code style="color:steelblue">'beds'</code>, <code style="color:steelblue">'sqft'</code>, and <code style="color:steelblue">'lot_size'</code>.
* **Tip:** If you want to plot multiple charts in the same code cell, remember to separate them by <code style="color:steelblue">plt.show()</code>, or else they will overlap.
* Among those features, do any look like they have potential outliers?

In [17]:
import warnings
warnings.filterwarnings('ignore')

In [18]:
# box plot of beds
sns.boxplot(df.beds, data=df)
plt.show()
# Violin plot of sqft
sns.boxplot(df.sqft, data=df)
plt.show()
# Violin plot of lot_size
sns.boxplot(df.lot_size, data=df)
plt.show()

TypeError: boxplot() got multiple values for argument 'data'

Among those three features, it looks like <code style="color:steelblue">lot_size</code> has a potential outlier! Look at its long and skinny tail. Let's look at the largest 5 lot sizes just to confirm.

#### B. Sort <code style="color:steelblue">df.lot_size</code> and display the top 5 samples.
* You can sort a Series with the <code style="color:steelblue">.sort_values()</code> function.
* **Tip:** <code style="color:steelblue">.sort_values()</code> has an argument called <code style="color:steelblue">ascending=</code>. By default, it's <code style="color:steelblue">True</code>
* **Tip:** You can chain functions together. This exercise can be completed in 1 line of code.

In [12]:
# Print top 5 homes with largest lot_size
df.lot_size.sort_values().tail()

1114    435600
1838    435600
1831    436035
1875    436035
1110    436471
Name: lot_size, dtype: int64

The largest property has a lot_size of 1,220,551 sqft. The next largest has a lot_size of only 436,471 sqft.

#### C. Display the entire row for that observation with the huge lot size.
* How many bedrooms and bathrooms does the property have?
* What's its property type?
* What's its transaction price?
* **Do you think this observation could be an outlier? Why?**

In [14]:
# print the record where lot_size = 1220551
#df.iloc[102]
df[df.lot_size = 1220551]

SyntaxError: cannot assign to attribute here. Maybe you meant '==' instead of '='? (2672710182.py, line 3)

#### D. Since all of the other observations are below 500,000 sqft, let's remove observations with <code style="color:steelblue">lot_size</code> greater than 500,000 sqft.
* You can use a boolean mask to do so (filter to keep only wanted observations).
* Overwrite the existing <code style="color:steelblue">df</code> object
* Print the length of the new <code style="color:steelblue">df</code> object.

In [16]:
# Drop the row with lot_size = 1220551
df = df.drop(102)

## IV. Save the Cleaned Dataset

#### Finally, save the cleaned dataframe as  <code style="color:steelblue">'cleaned_df.csv'</code>

Before we move on to the next module, let's save the new dataframe we worked hard to clean.
* We'll use Pandas's <code style="color:steelblue">.to_csv()</code> function.
* Set <code style="color:steelblue">index=<span style="color:crimson">None</span></code> so that Pandas drops the indices and only stores the actual data in the CSV.

In [24]:
# Save the cleaned dataset
df.to_csv('cleaned_df.csv', index=None)


We will use <code style="color:steelblue">'cleaned_df.csv'</code> in all the other modules!
In the next module, **Feature Engineering**, you'll use the cleaned dataset to create new features that will improve model performance.